# Test SVCNN Classification

In [27]:
from model.mvcnn import SVCNN
from modelnet40 import ModelNet40SingleView
import torch
import numpy as np
from torch.autograd import Variable

calc_accuracy = True

svcnn = SVCNN()
svcnn.load_state_dict(torch.load("pretrained/svcnn.pth"))
svcnn.eval()

val_dataset = ModelNet40SingleView("test", shuffle=False)
# indices = np.random.randint(low=0, high=val_dataset.__len__() ,size=100)
svcnn(val_dataset[1][1].unsqueeze(0))

if calc_accuracy:
    counter = 0
    for i in range(0, val_dataset.__len__()):
        sample = val_dataset[i][1]
        C,H,W = sample.size()
        in_data = Variable(sample).view(C,H,W)
        out_data = svcnn(in_data.unsqueeze(0))
        pred = torch.max(out_data, 1)[1]
        if val_dataset[i][0] == int(pred):
            counter += 1
        
    print(f"Accuaracy: {counter/val_dataset.__len__()*100:.5f}%") # 

Accuaracy: 90.63344%


# Test MVCNN Classification

In [30]:
from model.mvcnn import MVCNN
from modelnet40 import ModelNet40MultiView
import torch
import numpy as np
from torch.autograd import Variable

calc_accuracy = True

mvcnn = MVCNN()
mvcnn.load_state_dict(torch.load("pretrained/mvcnn.pth"))
mvcnn.eval()

val_dataset = ModelNet40MultiView("test", shuffle=False)
# indices = np.random.randint(low=0, high=val_dataset.__len__() ,size=100)

mvcnn(val_dataset[1][1])

if calc_accuracy:
    counter = 0
    for i in range(0, val_dataset.__len__()):
        sample = val_dataset[i][1]
        V,C,H,W = sample.size()
        in_data = Variable(sample).view(-1,C,H,W)
        out_data = mvcnn(in_data)
        pred = torch.max(out_data, 1)[1]
        if val_dataset[i][0] == int(pred):
            counter += 1
        
    print(f"Accuaracy: {counter/val_dataset.__len__()*100:.5f}%") # 

Accuaracy: 93.67909%


# Test PointNet Classification

In [2]:
from modelnet40 import ModelNet40AtlasNet
from hparams import HyperParameter
from model.network import BackboneNetwork
import torch

ATLAS_TEST = ModelNet40AtlasNet("test", shuffle=False, pcd_normalization="UnitBall",  
                                img_normalization=True, inference= True, num_point=2000)


hparams = HyperParameter()
hparams.load('pretrained/pointnet_hparams.yaml')
hparams.use_pre_trained_encoder = False
hparams.use_pre_trained_decoder = False

model = BackboneNetwork(hparams)
print("Loading Model ...")
model.load_state_dict(torch.load('pretrained/pointnet.pth',map_location='cpu'))
print("Model loaded.")
model.eval()

counter = 0
for i in range(0, ATLAS_TEST.__len__()):
    sample = ATLAS_TEST[i]['pointcloud'].float().transpose(1,2)
    out_data = model.forward(sample, classification=True)
    pred = torch.max(out_data, 1)[1]
    if int(pred) == ATLAS_TEST[i]['class_id']:
        counter+=1
        
print(f"PointNet Accuaracy: {counter/ATLAS_TEST.__len__()*100:.5f}%")

Encoder-PointNet initialized.
Every prim has 2000 points
New MLP decoder : hidden size 512, num_layers 2, activation relu
Decoder-AtlasNet initialized.
Loading Model ...
Model loaded.
PointNet Accuaracy: 88.89789%
